# 22. TODO: Finetuning SentenceTransformer

cf: https://sbert.net/docs/sentence_transformer/training_overview.html#

cf: https://sbert.net/docs/sentence_transformer/loss_overview.html

cf: https://github.com/math-collab/text-similarity/blob/main/SBert%20%2B%20fine-tune%20arXiv-class.ipynb

In [1]:
import pandas as pd

## Classfication Task: sentence pairs are either similar(1) or dissimilar (0)

## Anchor Task: a definition DA should be more similar to title TA than title TB

## Read Plagiasm Dataset from https://github.com/gipplab/Taxonomy-of-Mathematical-Plagiarism/tree/main/data

In [6]:
#df_plgsm = pd.read_json("data/annotated_spans.jsonl",lines=True)
del df_plgsm

In [7]:
import json
with open('data/annotated_spans.jsonl') as f:
    plag_data = [json.loads(line) for line in f]

In [8]:
plag_data[0]

{'srcDoc': {'srcMath': ['p5.m1', 'p5.m2', 'p5.m3', 'p5.m4', 'p5.m5'],
  'srcText': 'Furthermore, we denote by d\u2062σQ (resp. dSr) the (n-1)-dimensional volume elements induced by the Euclidean metric on ∂\u2061Cn\u2062(Ω) (resp. Sr )',
  'srcImages': [],
  'srcDocHTML': 'Furthermore, we denote by <math xmlns="http://www.w3.org/1998/Math/MathML" id="p5.m1" class="ltx_Math" display="inline"><mrow><mi>d</mi><mo>\u2062</mo><msub><mi>σ</mi><mi>Q</mi></msub></mrow></math> (resp. <math xmlns="http://www.w3.org/1998/Math/MathML" id="p5.m2" class="ltx_Math" display="inline"><mrow><mi>d</mi><msub><mi>S</mi><mi>r</mi></msub><mo>)</mo></mrow></math> the <math xmlns="http://www.w3.org/1998/Math/MathML" id="p5.m3" class="ltx_Math" display="inline"><mrow><mo stretchy="false">(</mo><mrow><mi>n</mi><mo>-</mo><mn>1</mn></mrow><mo stretchy="false">)</mo></mrow></math>-dimensional volume elements induced by the Euclidean metric on <math xmlns="http://www.w3.org/1998/Math/MathML" id="p5.m4" class="ltx_Ma

In [11]:
flattend = [{'srcText': doc['srcDoc']['srcText'], 'suspText':doc['suspDoc']['suspText'], 
             'srcDocName':doc['srcDoc']['srcDocName'], 'suspDocName': doc['suspDoc']['suspDocName'],
            'obfuscation':doc['casesReuse']['obfuscation']} for doc in plag_data]

In [13]:
df_flattened = pd.DataFrame(flattend)
df_flattened

,srcText,suspText,srcDocName,suspDocName,obfuscation
0,"Furthermore, we denote by d⁢σQ (resp. dSr) the...",and d⁢σQ is the surface area element on Sn(Ω))...,1252.31004.tex,1321.47163.tex,Paraphrase; Deletions
1,Theorem A Let g be a measurable function on ∂⁡...,The em A Let g be a measurable function on ∂⁡T...,1364.31004.tex,1379.35041.tex,Variation of subject; None; Different Presenta...
2,"Finally, a group G was said in [7] to be weakl...",Definition 2.2. A Q⁢T⁢A⁢G-module M is quasi tr...,1247.20062.tex,1344.16007.tex,Variation of subject; Different Presentation; ...
3,1 Introduction Let D be a domain in ℂ. Let ℱ b...,1 Introduction and results Let D be a domain i...,1338.35129.tex,1422.30056.tex,None; Paraphrase
4,"Corresponding to Theorem A, there are the foll...","Corresponding to Theorem A, there are the foll...",1338.35129.tex,1422.30056.tex,None; Paraphrase
...,...,...,...,...,...
1704,"In view of Corollary 4.4, a natural question i...",Although it is very interesting to note that t...,0168.27303.tex,1188.16002.tex,Variation of subject; Paraphrase
1705,"TheOrem 4.1. The group G=A¯+B, where A and B a...",Theorem 3.4. Let M be a Q⁢T⁢A⁢G-m⁢o⁢d⁢u⁢l⁢e su...,0168.27303.tex,1188.16002.tex,Variation of subject; Paraphrase; Deletions
1706,Corollary 2.2. The critical number of a primar...,Corollary 2.6. For a cobounded submodule N of ...,0168.27303.tex,1188.16002.tex,Variation of subject; Paraphrase; Formula Mani...
1707,THEOREM 2.1. Suppose that H is a cobounded sub...,Theorem 2.4. Let N be a cobounded submodule of...,0168.27303.tex,1188.16002.tex,Variation of subject; Paraphrase; Different Pr...


In [14]:
df_flattened.to_csv("data/plagiasm_span_pairs.csv", index=False)

In [15]:
df_flattened.describe()

,srcText,suspText,srcDocName,suspDocName,obfuscation
count,1709,1709,1709,1709,1709
unique,1662,1633,119,99,305
top,The Thom isomorphism theorem in equivariant K-...,"The virtual representation [Vλ,C]-[Wλ,C]∈R⁢(G~...",1247.20062.tex,1321.47163.tex,Variation of subject; Paraphrase
freq,3,3,67,106,174
